In [28]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 340 kB in 3s (133 kB/s)
Reading package lists... Done


In [29]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
# Import the time module so we can time our queries.
import time

# # Create a SparkSession
# spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

# Set the partitions to 4 or 8. 
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [30]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True, ignoreLeadingWhiteSpace=True)

# Show DataFrame
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

### Exploring data

In [31]:
# Print our schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [32]:
# Show the columns
df.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [33]:
# Describe our data
df.describe()

DataFrame[summary: string, id: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

### Adding a column for 'year_sold' from the date column

In [34]:
# Import date time functions
from pyspark.sql.functions import year

# Show the year for the date column
df.select(year(df["date"])).show(5)

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
+----------+
only showing top 5 rows



In [35]:
# Save the year from the date sold as a new column
df = df.withColumn("year_sold", year(df['date']))
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


### Question 2 - Creating a temporary view of the DataFrame

In [36]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

# Show DataFrame
spark.sql("SELECT (*) FROM home_sales").show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


### Question 3, 4, 5 answered using sql queries.

In [37]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""
SELECT year_sold, 
round(avg(price),2) as average_price
FROM home_sales 
WHERE bedrooms = 4 
group by year_sold
order by year_sold""").show()


+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [38]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
SELECT date_built, 
round(avg(price),2) as average_price
FROM home_sales 
WHERE bedrooms = 3 
and bathrooms =3 
group by date_built
order by date_built""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [39]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT date_built, 
round(avg(price),2) as average_price
FROM home_sales 
WHERE bedrooms = 3 
and bathrooms =3 
and floors = 2 
and sqft_living >= 2000 
group by date_built
order by date_built""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



### Question 6 with run time for the query recorded.
There was some debate about how to comprehend this question.
Bootcamp spot had: 
What is the "view" rating for homes costing more than or equal to $350,000?
Different to question in notebook. 

So I hope my interpretation is correct.

I have repeated question 6-15 using the bootcamp question below.

In [18]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, 
round(avg(price),2) as average_price 
FROM home_sales 
group by view
having average_price >= 350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  93|   1026006.06|
|  70|    695865.58|
|  67|    737970.96|
|  82|    1063498.0|
|  74|     745077.0|
|  64|    767036.67|
|  62|    759150.14|
|  86|   1070444.25|
|  51|    788128.21|
|  94|    1033536.2|
|  76|   1058802.78|
|  58|    759764.65|
|  57|     734340.5|
|  98|   1053739.33|
|  77|   1076205.56|
|  91|   1137372.73|
|  85|   1056336.74|
|  80|    991767.38|
|  72|    780914.67|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

--- 0.9112179279327393 seconds ---


In [40]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [41]:
# 8. Check if the table is cached.
# It will return True if it is cached.
spark.catalog.isCached('home_sales')

True

In [42]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, 
round(avg(price),2) as average_price 
FROM home_sales 
group by view
having average_price >= 350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  93|   1026006.06|
|  70|    695865.58|
|  67|    737970.96|
|  82|    1063498.0|
|  74|     745077.0|
|  64|    767036.67|
|  62|    759150.14|
|  86|   1070444.25|
|  51|    788128.21|
|  94|    1033536.2|
|  76|   1058802.78|
|  58|    759764.65|
|  57|     734340.5|
|  98|   1053739.33|
|  77|   1076205.56|
|  91|   1137372.73|
|  85|   1056336.74|
|  80|    991767.38|
|  72|    780914.67|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

--- 0.6685178279876709 seconds ---


In [43]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home_sales")
# df.write.parquet('parquet_home_sales', mode='overwrite')

In [44]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [45]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [46]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT view, 
round(avg(price),2) as average_price 
FROM p_home_sales 
group by view
having average_price >= 350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  70|    695865.58|
|  93|   1026006.06|
|  67|    737970.96|
|  82|    1063498.0|
|  74|     745077.0|
|  94|    1033536.2|
|  51|    788128.21|
|  86|   1070444.25|
|  62|    759150.14|
|  64|    767036.67|
|  58|    759764.65|
|  76|   1058802.78|
|  57|     734340.5|
|  77|   1076205.56|
|  98|   1053739.33|
|  85|   1056336.74|
|  91|   1137372.73|
|  72|    780914.67|
|  80|    991767.38|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

--- 0.5763449668884277 seconds ---


In [47]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [48]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False

### Question 6 repeated using bootcamp spot question.

In [49]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT round(avg(view),2) as average_view_rating 
FROM home_sales 
WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.6719672679901123 seconds ---


In [50]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [51]:
# 8. Check if the table is cached.
# It will return True if it is cached.
spark.catalog.isCached('home_sales')

True

In [52]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT round(avg(view),2) as average_view_rating 
FROM home_sales 
WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.22408080101013184 seconds ---


In [53]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home_sales")
# df.write.parquet('parquet_home_sales', mode='overwrite')

In [54]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [55]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [56]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT round(avg(view),2) as average_view_rating 
FROM p_home_sales 
WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.43297719955444336 seconds ---


In [57]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [58]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False